# **핵심 키워드 추출**

## 1 TF-IDF 활용 핵심 키워드 추출

### 실습1. sklearn 활용

In [1]:
import requests
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url)
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select('h3#articleTitle')[0].text  # 제목
    content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ")  # 본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return  content.strip()

docs = []
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191'))
docs.append(get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728'))
len(docs)

5

#### 1) 전처리

In [2]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs:
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

#### 2) TF-IDF 계산

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector = count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '구축', '민간']

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

#### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x : (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [feature_names[idx], score]